# 02 — Query Demo (Answer from Context + Citations)
Ask a question like 'Where did we define precision vs. recall?' and get (deck, page) cites.

## Load store & model (auto-CUDA on Deck)

In [ ]:
import os

# Tell Hugging Face to skip TensorFlow/Flax so they never import TensorFlow (TF).
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["TRANSFORMERS_NO_FLAX"] = "1"

# Quiet TF logs if something still pulls it in.
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # 1=INFO, 2=WARNING, 3=ERROR


## Search function

Save / reload FAISS index and all metadata so that later so we don't need re-embed.


In [ ]:
import json, os
import faiss
from pathlib import Path

# make a directory  to store FAISS data.
# exist_ok=True --> prevents it from crashing if it exists
os.makedirs("data/faiss", exist_ok=True)

# Saves the FAISS index into a file. This is the vector store (embeddings) :)
faiss.write_index(index, "data/faiss/canvas.index")

# opens a JSON file to write in and puts in facts(the text chuncks we built earlier) and metas (the metadata)
with open("data/faiss/facts.json", "w", encoding="utf-8") as f:
    json.dump({"facts": facts, "metas": metas}, f, ensure_ascii=False) # Persistence: save / load FAISS store + metadata


In [ ]:
# Defining paths where the FAISS data will be located
STORE_DIR  = f"{BASE}/data/faiss"
INDEX_PATH = f"{STORE_DIR}/canvas.index"
FACTS_PATH = f"{STORE_DIR}/facts.json"


In [ ]:
# makes sure the folder exits
# saves the FAISS index and the facts/metadata
# then prints a confirmation
def save_store(index, facts, metas, store_dir=STORE_DIR):
    Path(store_dir).mkdir(parents=True, exist_ok=True)
    faiss.write_index(index, os.path.join(store_dir, "canvas.index"))
    with open(os.path.join(store_dir, "facts.json"), "w", encoding="utf-8") as f:
        json.dump({"facts": facts, "metas": metas}, f, ensure_ascii=False)
    print(" saved:", INDEX_PATH, "and", FACTS_PATH)



# loads the vector index back into memory
# opens the JSON file and loaads the facts and metadata
# returns them to allow for querying without having to recompute the embeddings
def load_store(store_dir=STORE_DIR):
    idx = faiss.read_index(os.path.join(store_dir, "canvas.index"))
    with open(os.path.join(store_dir, "facts.json"), "r", encoding="utf-8") as f:
        data = json.load(f)
    print(" loaded:", os.path.join(store_dir, "canvas.index"), "and facts.json")
    return idx, data["facts"], data["metas"]

# This saves preceding index, facts, and metadata right after building
#save_store(index, facts, metas)

In [ ]:

# Reload in a fresh session--Refresh style
# index, facts, metas = load_store()
# route_emb = {k: model.encode([v], normalize_embeddings=True).astype("float32") for k,v in ROUTE_DESC.items()}


# We'll use this to reload:
# index = faiss.read_index("data/faiss/canvas.index")
# facts_meta = json.load(open("data/faiss/facts.json","r",encoding="utf-8"))
# facts, metas = facts_meta["facts"], facts_meta["metas"]
